# Реализовать на С или python поиск минимального значения для каждой из 5 заданных двумерных функций f(x,y) на квадрате 0 ⩽ x,y ⩽ 10:
(a) fa(x,y)=(x−3)2+y2+1
(b) fb(x,y)=(x+1)2+(2y−8)2+1
(c) fc(x,y)=x2+xy+y2−16x−17y+94
(d) fd(x,y)=((x−7)2+(y−8)2)·((x−8)2+(y−7)2)+4
(e) fe(x,y)=(4xy−19)2·(cos2πx+cos2πy)−x−y+24
# Решение вычислить с точностью ε = 10−6.
# В качестве начальной точки взять (x0, y0) = (1, 2).
# Найти точку минимума (x∗,y∗), значение функции f(x∗,y∗) и количество итераций (вычислений функции) n, которое потребовалось на достижение требуемой точности выбранным методом.


In [20]:
import math
import numpy as np
import matplotlib.pyplot as plt

globalEpsilon = 1e-6
radius = 8                                 # working plane radius
# centre = (1, 2)                          # very, very bad point)
centre = (globalEpsilon, globalEpsilon)    # centre of the working circle
arrShape = 100                             # number of points processed / 360
step = radius / arrShape                   # step between two points


def firstFunction(x, y):
    return (x - 3) * (x - 3)  + y * y + 1

def secondFunction(x, y):
    return (x + 1)*(x + 1) + (2*y - 8)*(2*y - 8) + 1

def thirdFunction(x, y):
    return x * x + (x * y) + y * y - (16 * x) - (17 * y) + 94

def fourthFunction(x, y):
    return ((x - 7) * (x - 7) + (y - 8) * (y - 8)) * ((x - 8) * (x - 8) + (y - 7) * (y - 7)) + 4

def fifthFunction(x, y):
    return (4 * x * y - 19) * (4 * x * y - 19) * (math.cos(2 * x * math.pi) + math.cos(2 * y * math.pi)) - x - y+24

Будем использовать метод градиентного спуска

In [32]:
def rotateVector(length, a):
    return length * np.cos(a), length * np.sin(a)

def calculateFlipPoints(customFunction):
    flipPoints = np.array([[0, 0], [0, 0]])
    points = np.zeros((360, arrShape), dtype=bool)
    cx, cy = centre

    for i in range(arrShape):
        for alpha in range(360):
            x, y = rotateVector(step, alpha)
            x = x * i + cx
            y = y * i + cy
            points[alpha][i] = derivativeX(x, y, customFunction) + derivativeY(y, x, customFunction) > 0
            if not points[alpha][i - 1] and points[alpha][i]:
                flipPoints = np.vstack((flipPoints, np.array([alpha, i - 1])))

    return flipPoints

def pickEstimates(positions, customFunction):
    vx, vy = rotateVector(step, positions[1][0])
    cx, cy = centre
    bestX, bestY = cx + vx * positions[1][1], cy + vy * positions[1][1]

    for index in range(2, len(positions)):
        vx, vy = rotateVector(step, positions[index][0])
        x, y = cx + vx * positions[index][1], cy + vy * positions[index][1]
        if customFunction(bestX, bestY) > customFunction(x, y):
            bestX = x
            bestY = y

    for index in range(360):
        vx, vy = rotateVector(step, index)
        x, y = cx + vx * (arrShape - 1), cy + vy * (arrShape - 1)
        if customFunction(bestX, bestY) > customFunction(x, y):
            bestX = x
            bestY = y

    return bestX, bestY

def derivativeY(epsilon, arg, customFunction):
    return (customFunction(arg, epsilon + globalEpsilon) - customFunction(arg, epsilon)) / globalEpsilon

def derivativeX(epsilon, arg, customFunction):
    return (customFunction(globalEpsilon + epsilon, arg) - customFunction(epsilon, arg)) / globalEpsilon

def gradient(x, y, customFunction):
    return derivativeX(x, y, customFunction) + derivativeY(y, x, customFunction)

def gradientDescent(bestEstimates, isX, customFunction):
    derivative = derivativeX if isX else derivativeY
    bestX, bestY = bestEstimates
    descentStep = step
    value = derivative(bestY, bestX, customFunction)

    while abs(value) > globalEpsilon:
        descentStep *= 0.95
        bestY = bestY - descentStep \
            if derivative(bestY, bestX, customFunction) > 0 else bestY + descentStep
        value = derivative(bestY, bestX, customFunction)

    return bestY, bestX

In [4]:
def findMinimum(customFunction):
    return gradientDescent(gradientDescent(pickEstimates(calculateFlipPoints(customFunction), customFunction), False, customFunction), True, customFunction)

In [6]:
def getGrid(grid_step, customFunction):
    samples = np.arange(-radius, radius, grid_step)
    x, y = np.meshgrid(samples, samples)
    return x, y, customFunction(x, y)

def drawChart(point, grid):
    pointX, pointY, pointZ = point
    gridX, gridY, gridZ = grid
    plt.rcParams.update({
        'figure.figsize': (4, 4),
        'figure.dpi': 200,
        'xtick.labelsize': 4,
        'ytick.labelsize': 4
    })
    ax = plt.figure().add_subplot(111, projection='3d')
    ax.scatter(pointX, pointY, pointZ, color='red')
    ax.plot_surface(gridX, gridY, gridZ, rstride=5, cstride=5, alpha=0.7)
    plt.show()

In [30]:
myFunction = firstFunction
minX, minY = findMinimum(myFunction)
minimum = (minX, minY, myFunction(minX, minY))
print("Function min = ", minimum)
# drawChart(minimum, getGrid(0.05, myFunction))

IndexError: invalid index to scalar variable.